In [1]:
import os
import pandas as pd
from wordcloud import WordCloud
import copy
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from kiwipiepy import Kiwi
kiwi = Kiwi(typos='basic')
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_국민의힘 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df국민의힘.csv')
df_날씨 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df날씨.csv')
df_민주당 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df민주당.csv')
df_의대 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df의대.csv')
df_전쟁 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df전쟁.csv')
df_정부 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df정부.csv')

df_구글 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df구글.csv')
df_금리 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df금리.csv')
df_네이버 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df네이버.csv')
df_부동산 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df부동산.csv')
df_삼성 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df삼성.csv')
df_애플 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df애플.csv')
df_주식 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전처리데이터//df주식.csv')

news_df = pd.concat([df_국민의힘,df_날씨,df_민주당,df_의대,df_전쟁,df_정부, df_구글, df_금리, df_네이버, df_부동산,df_삼성,df_애플,df_주식],ignore_index=True )

### 국민의힘: 0~1645  날씨: 1646~3610  민주당:  3611~5441  의대:  5442~7797  전쟁: 7798~9263  정부: 9264~12233 구글: 12234~12982  
### 금리: 12983~14543 네이버: 14544~15495 부동산: 15496~16938  삼성: 16939~18993 애플: 18994~19695 주식: 19696~20988

# Body tf-idf

In [72]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed

# 전처리된 문서 리스트
preprocessed_documents = news_df['preprocessed_body']

# TF-IDF 벡터라이저 초기화 및 문서 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)

def compute_similarity(doc_index):
    # 특정 문서와 다른 문서 간의 코사인 유사도 계산
    cosine_similarities = cosine_similarity(tfidf_matrix[doc_index], tfidf_matrix).flatten()
    return cosine_similarities

def get_similar_documents(doc_index, top_n=10):
    # 특정 문서에 대한 코사인 유사도 계산
    cosine_similarities = compute_similarity(doc_index)

    # 유사도가 높은 순서대로 문서 정렬 (자신 제외)
    similar_docs_indices = np.argsort(-cosine_similarities)[1:top_n+1]
    
    print(f"문서 {doc_index}: {news_df['title'][doc_index]} --- 와 유사한 문서들:")
    for idx in similar_docs_indices:
        print(f"문서 {idx}: {news_df['title'][idx]} (유사도: {cosine_similarities[idx]:.3f})")

# 예제: n번째 문서와 유사한 10개의 문서 찾기
doc_index = int(input('인덱스 번호를 입력하세요: '))
get_similar_documents(doc_index, top_n=10)


문서 20741: [미장브리핑] 28일부터 미국 증권거래 결제일 T+1로 --- 와 유사한 문서들:
문서 19731: 5월 28일부터 미국 주식 '주문 다음 날' 결제된다 (유사도: 0.507)
문서 19797: 이달말 美주식 'T+1일 결제'…예탁원 "국내 주식 거래 큰 영향 없어" (유사도: 0.502)
문서 19722: 미국 주식시장 결제일 28일부터 'T+1'로 단축 (유사도: 0.498)
문서 19926: 美 결제주기 T+1로 단축..."28일 이행준비 완료" (유사도: 0.494)
문서 19751: 이달말 美주식 'T+1일 결제'…예탁원, 업무방식·전산제도 개선 (유사도: 0.482)
문서 19721: 미국 주식 결제주기, 28일부터 'T+1일'로 하루 앞당겨진다 (유사도: 0.475)
문서 19808: 예탁원, ‘美주식 결제주기 T+1 단축’ 막바지 준비 (유사도: 0.470)
문서 19704: 28일부터 美 주식 거래 다음 날 돈 들어온다… 예결원 “준비 완료” (유사도: 0.470)
문서 20763: 월요일에 엔비디아 주식 팔면 수요일에 입금…결제주기 단축 여파는 (유사도: 0.466)
문서 19757: 美주식 결제주기 1일 단축…예결원, 국내 증권사와 대응책 마련 (유사도: 0.465)


문서 20741: [미장브리핑] 28일부터 미국 증권거래 결제일 T+1로 --- 와 유사한 문서들:
문서 19722: 미국 주식시장 결제일 28일부터 'T+1'로 단축 (유사도: 0.422)
문서 10769: 미장브리핑 美정부 비축유 100만배럴 푼다 (유사도: 0.415)
문서 20247: 28일부터 미국 주식 결제일 하루 앞당겨져…“미수거래 주의해야” (유사도: 0.400)
문서 14354: [미장브리핑] 메모리얼데이 휴장…ECB 6월 금리 인하 가능성↑ (유사도: 0.369)
문서 19978: 美 주식시장 결제일 28일부터 'T+1'로 단축 (유사도: 0.330)
문서 20228: 하루 앞당겨지는 미국 주식 결제일…투자자 유의점은? (유사도: 0.32

# Title tf-idf

### 국민의힘: 0~1645  날씨: 1646~3610  민주당:  3611~5441  의대:  5442~7797  전쟁: 7798~9263  정부: 9264~12233 구글: 12234~12982  
### 금리: 12983~14543 네이버: 14544~15495 부동산: 15496~16938  삼성: 16939~18993 애플: 18994~19695 주식: 19696~20988

In [42]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed

# 전처리된 문서 리스트
preprocessed_documents = news_df['preprocessed_title']

# TF-IDF 벡터라이저 초기화 및 문서 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)

def compute_similarity(doc_index):
    # 특정 문서와 다른 문서 간의 코사인 유사도 계산
    cosine_similarities = cosine_similarity(tfidf_matrix[doc_index], tfidf_matrix).flatten()
    return cosine_similarities

def get_similar_documents(doc_index, top_n=10):
    # 특정 문서에 대한 코사인 유사도 계산
    cosine_similarities = compute_similarity(doc_index)

    # 유사도가 높은 순서대로 문서 정렬 (자신 제외)
    similar_docs_indices = np.argsort(-cosine_similarities)[1:top_n+1]
    
    print(f"문서 {doc_index}: {news_df['title'][doc_index]} --- 와 유사한 문서들:")
    for idx in similar_docs_indices:
        print(f"문서 {idx}: {news_df['title'][idx]} (유사도: {cosine_similarities[idx]:.3f})")

# 예제: n번째 문서와 유사한 10개의 문서 찾기
doc_index = int(input('인덱스 번호를 입력하세요: '))
get_similar_documents(doc_index, top_n=10)


문서 3015: 오늘의 날씨울산26일 일늦은 오후부터 강풍 동반 비 --- 와 유사한 문서들:
문서 1734: 오늘의 날씨울산15일 수늦은 오후부터 강풍 동반 비 (유사도: 1.000)
문서 1759: 오늘의 날씨 부산경남14일 수늦은 오후부터 강풍 동반 비 (유사도: 0.692)
문서 3336: 오늘의 날씨울산28일 화아침 기온 뚝 강풍주의 (유사도: 0.565)
문서 3024: 오늘의 날씨울산27일 월강풍 조심 낮 최고 26도 (유사도: 0.563)
문서 3205: 오늘의 날씨울산25일 토오후부터 맑음 낮 최고 21도 (유사도: 0.555)
문서 1832: 날씨 전국에 강풍 동반한 비언제까지 오나 (유사도: 0.547)
문서 3164: 날씨 오늘 오후부터 비곳곳 돌풍벼락 동반 (유사도: 0.497)
문서 1771: 오늘의 날씨 광주전남15일 수오후 5 비강풍주의 (유사도: 0.491)
문서 2328: 오늘의 날씨 울산18일 토낮 최고 27도 일교차 커 (유사도: 0.471)
문서 2623: 오늘의 날씨울산22일 수낮 최고 25도 일교차 커 (유사도: 0.471)
